**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

In [ ]:
%pip install -q ipylab

# Ipylab

`Ipylab` is the primary class that provides functionality for scheduling and awaiting operations between Python objects and the corresponding model in the frontend. Each operation is run as a Task which is assigned a UUID and sent as a custom message. 

`IpylabModel` has a `base` object that is set prior to being *ready*. The base object is specified by the Python class and located by the frontend model with the `ipylabInit` method (customised by other subclasses). 

## Operations

The `operation` method is the workhorse through which all asynchronous messaging is done. It will always return a task. Each operation much be implement in the frontend model. 


## Generic methods

 `Ipylab` implements the following generic methods interact with objects in the fronted: 

* execute_method
* get_property
* set_property
* update_property
* list_properties

These methods pass an object to utility functions reducing the need to write corresponding code in the frontend. 

By default the methods the `base`. A different base object can be specified `obj=Obj.###` to specify a different path.



## Connection

A `Connection` is subclassed from `Ipylab` providing a connection to an object in the Frontend. 

Each connection has a unique `cid` (stands for connection id).  The `cid` consists of the class prefix `ipylab-<CLASS NAME>|` followed by one or more parts joined with a pipe. The class prefix part of the cid determines the type of class that is created when creating a new instance.

In the kernel there is only one instance of a Connection object per `cid`. For example: calling `Connection('ipylab-Connection|my cid')` multiple times will get the same object inside the same kernel.

The object in the frontend is stored as an ObservableDisposable and once it has be set, it cannot be replaced with another object, unless it has been disposed.

Calling the close method in Python will by default dispose of the object in the frontend. This can behavior can be changed by using `dispose=False`, or previously setting the 'auto_dispose` trait to False.


### Subclasses

Subclasses of `Connection` are identified by its *prefix* `.

### Connection models

There are two models for connections, `ConnectionModel` and `ShellConnectionModel`. `ShellConnectionModel` is specific for widgets that are loaded in the shell.

### Making a connection

Many of the builtin methods return connections.

The easiest way to create a connection is to specify the `transform` as a `Transform.connection`. Examples:

* [Autostart](plugins.ipynb#Example-launching-a-small-app)
* [Console](http://localhost:9999/lab/tree/commands.ipynb#Create-a-new-console)
* [Notebook](sessions.ipynb#Create-a-new-Notebook)

Note: This notebook should be run one line at a time waiting for each cell to return before running the next cell.

In [ ]:
import ipylab

app = ipylab.App()

t = app.list_properties("shell", depth=2)

In [ ]:
methods = t.result()
methods

Some of the methods above have already been implemented in 'app.shell' on the Python object. And if you look at the code you'll see these methods use the *generic* method `execute_method`.

In case you're interested, the custom message sent to the frontend looks like this:

```python
{
    "method": "custom",
    "content": '{"ipylab_PY": "353695a0-1754-4d53-b758-291f70058f41", "operation": "listProperties", "kwgs": {"dottedname": "shell", "depth": 2, "omitHidden": true}, "transform": "raw"}',
}
```

Note the *content* is always JSON string. This is because we convert every message to JSON prior to sending making it possible to replace widgets and code objects with string representations. For widgets, it is necessary to instruct the frontend what to do with the widget. Currently there are the following additional options:
1. `toLuminoWidget`: Replace with a widget in the frontend.
2. `toObject`: Replace with an object in the frontend.

These options are lists of attributes as they appear in the frontend that should be transformed/extracted.

In [ ]:
# The source code

import inspect

print(inspect.getsource(app.shell.expand_left))  # noqa: T201

In [ ]:
t = app.shell.expand_right()  # Built in

In [ ]:
t = app.shell.execute_method("collapseRight")  # Generic

Note that all Python methods are written with `snake_case` whereas the Javascript command is written as `cammelCase`.